In [ ]:
from kafka import KafkaConsumer, KafkaProducer
from kafka.admin import NewTopic
import json
import time
from datetime import datetime
import pandas as pd

In [ ]:
BOOTSTRAP_SERVERS = ['localhost:9092', 'localhost:9093', 'localhost:9094']

In [ ]:
producer = KafkaProducer(
    bootstrap_servers=BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    key_serializer=lambda v: json.dumps(v).encode('utf-8')
)

def send_message():
    try:
        # Topic để test
        topic_name = "test-topic"
        
        while True:
            # Tạo message test
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            message = {
                "timestamp": current_time,
                "message": f"Test message at {current_time}",
                "value": round(time.time())
            }
            
            # Key cho message
            key = {"key": f"key_{round(time.time())}"}
            
            # Gửi message
            future = producer.send(topic_name, key=key, value=message)
            
            # Đợi để đảm bảo message được gửi thành công
            record_metadata = future.get(timeout=10)
            
            print(f"\nMessage sent successfully:")
            print(f"Topic: {record_metadata.topic}")
            print(f"Partition: {record_metadata.partition}")
            print(f"Offset: {record_metadata.offset}")
            print(f"Message: {message}")
            
            # Đợi 5 giây trước khi gửi message tiếp
            time.sleep(5)
            
    except Exception as e:
        print(f"Error sending message: {e}")
    finally:
        producer.close()

In [ ]:
BOOTSTRAP_SERVERS = ['localhost:9092', 'localhost:9093', 'localhost:9094']
cosumer = KafkaConsumer(bootstrap_servers=BOOTSTRAP_SERVERS)
cosumer.topics()

In [ ]:

consumer_2 = KafkaConsumer(
    'streaming.public.yellow_trip_raw',  # tên topic
    bootstrap_servers=BOOTSTRAP_SERVERS,
    auto_offset_reset='earliest'
)

In [ ]:
for mess in consumer_2:
    print(mess.value)
    print(mess.value)
    break

In [ ]:
for mess in cosumer:
    print(mess)

In [ ]:
import time
import logging
from bson import json_util
def create_topic(admin, topic_name):
    """
        Create topic if not exists
    """
    try:
        topic = NewTopic(name=topic_name, num_partitions=1, replication_factor=1)
        admin.create_topic([topic])
        logging.info(f"A new topic {topic_name} has been created!")
    except Exception:
        logging.info(f"Topic {topic_name} already exists. Skipping creation!")
        pass

def format_record(row):
    taxi_res = {}
    column_names = [
        'dolocationid', 'pulocationid', 'ratecodeid', 'vendorid',
        'congestion_surcharge', 'extra', 'fare_amount', 'improvement_surcharge',
        'mta_tax', 'passenger_count', 'payment_type', 'tip_amount',
        'tolls_amount', 'total_amount', 'dropoff_datetime', 'pickup_datetime',
        'trip_distance'
    ]
    for i, column_name in enumerate(column_names):
        if 'datetime' in column_name:
            taxi_res[column_name] = str(row.iloc[i])
        else:
            taxi_res[column_name] = row.iloc[i]
    return taxi_res


def streaming_data(servers, DATA_PATH):
    producer = None
    admin = None
    for _ in range (10): 
        try:
            producer = KafkaProducer(bootstrap_servers=servers)
            admin = KafkaAdminClient(bootstrap_servers=servers)
            break
        except Exception as e:
            logging.info(f"Trying to instantiate admin and producer with bosootstrap server {servers} with error {e}")
            time.sleep(10)
            pass
    
    df = pd.read_parquet(DATA_PATH)
    
    for index, row in df.iterrows():
        topic_name = "nyc_taxi_trip"
        create_topic(admin, topic_name=topic_name)
        producer.send(
            topic_name, json.dumps(format_record(row), default=json_util.default).encode('utf-8')
        )
        print(f"Sent record {index} to topic {topic_name}")
        time.sleep(2)
        
    

In [ ]:
from kafka import KafkaConsumer
BOOTSTRAP_SERVERS = ['localhost:9092']
topic = "test-topic"

consumer = KafkaConsumer(
    topic,
    bootstrap_servers=BOOTSTRAP_SERVERS,
    auto_offset_reset='earliest',
)
consumer

In [ ]:
for message in consumer:
    print(message.value)

In [8]:
consumer.close()

In [ ]:
topic_streaming_1 = "streaming.public.yellow_trip_raw"
topic_streaming_2 = "streaming.public.green_trip_raw"
consumer_2 = KafkaConsumer(
    topic_streaming
)

In [ ]:
consumer.topics()